In [1]:
from populate_decimation import *

Connecting celiib@10.28.0.34:3306


In [2]:
external_decimated_mesh_path

'/mnt/dj-stor01/platinum/minnie65/01/decimated_meshes'

In [4]:
filename = '96631955273149705_0.h5'

In [5]:
#f'{segment_id}_{version}.h5'

In [7]:
filepath = os.path.join(external_decimated_mesh_path, filename)

In [8]:
os.path.exists(filepath)

True

In [9]:
minnie.Decimation & dict(segment_id=96631955273149705)

segment_id segment id unique within each Segmentation,version,decimation_ratio ratio of remaining mesh vertices/faces (depends on what metric the decimation technique uses),n_vertices,n_faces,mesh in-place path to the hdf5 (decimated) mesh file
96631955273149705,0,0.25,395000,790450,=BLOB=


In [11]:
if len(minnie.Decimation & dict(segment_id=96631955273149705)) == 1:
    print('exists')

exists


In [7]:
def concatenated_rel(cls, core_segment=None, version=-1, return_with_meshes=False):
    """
    Returns all. You can restrict by a core_segment first though.
    
    :core_segment: The core segment to restrict by. If left empty will fetch all.
    :version: The default will fetch the highest version for each core segment
        and its subsegments.
    :param return_with_meshes: When set to true or 'Decimation' will default
        to using the Decimation table for the meshes, otherwise 'Mesh' will
        choose the Mesh table with the original meshes.
    """
    
    subsegment_rel = cls.Subsegment.proj()

    if core_segment is not None:
        subsegment_rel &= dict(segment_id=core_segment)

    if version == -1:
        version_rel = dj.U('segment_id').aggr(subsegment_rel, version='max(version)')
    else:
        version_rel = subsegment_rel & dict(version=version)

    a_rel = dj.U('segment_id') & version_rel
    b_rel = dj.U('segment_id') & subsegment_rel.proj(_='segment_id', segment_id='subsegment_id')
    c_rel = a_rel + b_rel

    if return_with_meshes:
        if isinstance(return_with_meshes, str) and return_with_meshes.lower() == 'mesh':
            c_rel = m65.Mesh & c_rel
        else:
            c_rel = m65.Decimation & c_rel
    
    return c_rel


def get_neuromancer_seg_ids(segment_id):
    return concatenated_rel(minnie.FromNeuromancer, segment_id).fetch('segment_id')

In [9]:
core_segments = (dj.U('segment_id') & minnie.FromNeuromancer).fetch('segment_id')

In [13]:
version = 0
decimation_ratio = 0.25 # Only the value in the .mls can change the ratio though.

for core_segment in core_segments:
    seg_ids = get_neuromancer_seg_ids(core_segment)
    for segment_id in seg_ids:
        key = {'segment_id': segment_id}
        mesh = (minnie.Mesh & key).fetch1('mesh')

        new_vertices, new_faces = decimate_mesh(mesh.vertices, mesh.faces, key['segment_id'], folder_name)
        Decimation.make_entry(
            segment_id=key['segment_id'],
            version=version,
            decimation_ratio=decimation_ratio,
            vertices=new_vertices,
            faces=new_faces
        )

xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i decimation_temp/neuron_96631955273149705.off -o decimation_temp/neuron_96631955273149705_decimated.off -s /notebooks/Platinum_Blender/decimation_meshlab.mls


DataJointError: Inserts into an auto-populated table can only done inside its make method during a populate call. To override, set keyword argument allow_direct_insert=True.